In [73]:
from huggingface_hub import notebook_login

In [74]:
notebook_login()

In [75]:
!pip install accelerate peft bitsandbytes trl py7zr auto-gptq optimum

In [76]:
!pip install transformers

In [77]:
from datasets import load_dataset,Dataset

In [78]:
data = load_dataset("samsum",split="train")

In [79]:
data_df = data.to_pandas()

In [80]:
data_df

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [81]:
#merge dialogue and summary(combined assitant and model response)
data_df["text"]=data_df[["dialogue","summary"]].apply(lambda x: "###Human: Summarize the following dialogue: " + x["dialogue"] + "\n###Assistant: "+x["summary"],axis=1)

In [82]:
data_df["text"]

,text
0,###Human: Summarize the following dialogue: Am...
1,###Human: Summarize the following dialogue: Ol...
2,###Human: Summarize the following dialogue: Ti...
3,###Human: Summarize the following dialogue: Ed...
4,###Human: Summarize the following dialogue: Sa...
...,...
14727,###Human: Summarize the following dialogue: Ro...
14728,###Human: Summarize the following dialogue: Th...
14729,###Human: Summarize the following dialogue: Jo...
14730,###Human: Summarize the following dialogue: Je...


In [83]:
data_df["text"][0]

"###Human: Summarize the following dialogue: Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)\n###Assistant: Amanda baked cookies and will bring Jerry some tomorrow."

In [84]:
data = Dataset.from_pandas(data_df)

In [85]:
data

Dataset({
    features: ['id', 'dialogue', 'summary', 'text'],
    num_rows: 14732
})

In [86]:
from peft import LoraConfig,get_peft_model,prepare_model_for_kbit_training,AutoPeftModelForCausalLM

In [91]:
 # Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [92]:
tokenizer.pad_token = tokenizer.eos_token

In [93]:
quantization_config=GPTQConfig(bits=4,disable_exllama=True,tokenizer=tokenizer)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


In [94]:
model = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",quantization_config=quantization_config,device_map="auto")

config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.1-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [95]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)

In [96]:
Lora_config = LoraConfig(r=16,lora_alpha=16,target_modules=["q_proj","v_proj"],lora_dropout=0.05,bias="none",task_type="CAUSAL_LM")

In [97]:
model = get_peft_model(model,Lora_config)

In [98]:
device="cuda"

In [99]:
device

'cuda'

In [100]:
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (rotary_emb): MistralRotaryEmbedding()
              (k_proj): QuantLinear()
              (o_proj): QuantLinear()
              (q_proj): lora.QuantLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Par

In [101]:
from trl import SFTTrainer

In [102]:
trainingArs = TrainingArguments(output_dir="Mistral-7B-Instruct-v0.2-GPTQ-finetuned-samsum",
                                per_device_train_batch_size=4, # Reduced from 8
                                gradient_accumulation_steps=2, # Increased from 1
                                optim="paged_adamw_32bit",
                                learning_rate=2e-4,
                                lr_scheduler_type="cosine",
                                save_strategy="epoch",
                                logging_steps=100,
                                num_train_epochs=1,
                                max_steps=250,
                                fp16=False, # Consider setting this to False if batch size reduction is not enough
                                push_to_hub=False,
                                gradient_checkpointing=True)

In [103]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Set memory fraction for a process
    torch.cuda.set_per_process_memory_fraction(0.9)
else:
    print("CUDA is not available. Skipping memory fraction setting.")

In [104]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [105]:
for param in stfTrainer.model.base_model.model.model.embed_tokens.parameters():
    param.requires_grad = True

In [106]:
stfTrainer = SFTTrainer(model=model,
                         train_dataset=data,
                         peft_config=Lora_config,
                         dataset_text_field="text",
                         max_seq_length=256, # Reduce if possible
                         tokenizer=tokenizer,
                         args=trainingArs,
                        packing=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [108]:
for name, param in stfTrainer.model.named_parameters():
    param.requires_grad = True

stfTrainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
100,2.556500
200,2.546100


TrainOutput(global_step=250, training_loss=2.548968780517578, metrics={'train_runtime': 2233.7699, 'train_samples_per_second': 0.895, 'train_steps_per_second': 0.112, 'total_flos': 402611655868416.0, 'train_loss': 2.548968780517578, 'epoch': 0.13575889220743959})

In [118]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGr

In [122]:
stfTrainer.push_to_hub("smd29/mistral-7b-gptq-chat-finetuned")

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

events.out.tfevents.1724906911.76121a501529.1134.0:   0%|          | 0.00/5.86k [00:00<?, ?B/s]

events.out.tfevents.1724906954.76121a501529.1134.1:   0%|          | 0.00/5.86k [00:00<?, ?B/s]

events.out.tfevents.1724907850.76121a501529.1134.2:   0%|          | 0.00/5.86k [00:00<?, ?B/s]

Upload 13 LFS files:   0%|          | 0/13 [00:00<?, ?it/s]

...

events.out.tfevents.1724909973.76121a501529.1134.9:   0%|          | 0.00/12.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

events.out.tfevents.1724909199.76121a501529.1134.8:   0%|          | 0.00/5.86k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/smd29/Mistral-7B-Instruct-v0.2-GPTQ-finetuned-samsum/commit/af2557e844eefd7541be14d9f2affa80182dd5ab', commit_message='smd29/mistral-7b-gptq-chat-finetuned', commit_description='', oid='af2557e844eefd7541be14d9f2affa80182dd5ab', pr_url=None, pr_revision=None, pr_num=None)

In [123]:
tokenizer.push_to_hub("smd29/mistral-7b-gptq-chat-finetuned")

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/smd29/mistral-7b-gptq-chat-finetuned/commit/d1855f814159da26e46a720887f131f510c7c718', commit_message='Upload tokenizer', commit_description='', oid='d1855f814159da26e46a720887f131f510c7c718', pr_url=None, pr_revision=None, pr_num=None)